# 第4回：医療AIの評価指標

## 学習目標

- ROC曲線とAUCの意味を理解し、実際のモデルで描画できるようになる
- 感度・特異度のトレードオフと閾値設定の考え方を学ぶ
- 医療における「許容できる誤り」と評価指標の選択を議論する

---
## はじめに

**操作方法**: `Shift + Enter` でセルを順番に実行してください。セッションが切断された場合は、上から再実行すれば復旧できます。

---

### この講義で挑戦すること

AIが「精度95%」と言われたら、信頼できますか？

実は「精度」だけでは、AIの性能を正しく評価できません。例えば、100人中95人が健康なデータで「全員健康」と予測すれば、精度95%を達成できます。しかし、病気の5人を全員見逃してしまいます。

この講義では、医療AIを正しく評価するための指標を学びます。**感度・特異度・AUC**といった指標を理解し、「どの閾値で判定すべきか」を自分で決められるようになりましょう。

---
## 概念学習：ROCシミュレータで体験（推奨）

実際のコードに入る前に、以下のインタラクティブシミュレータで概念を体験することをお勧めします。

**[ROC/AUC Educational Simulator](https://kshimoji8.github.io/hds-lab/roc-simulator.html)**

（上のリンクをクリックするとブラウザで直接開けます）

### シミュレータで試してほしいこと

1. **分離度を変えてAUCの変化を観察**
   - 分離度0（完全に重なる）→ AUC ≈ 0.5
   - 分離度100（完全に分離）→ AUC ≈ 1.0

2. **閾値を動かして感度・特異度のトレードオフを体感**
   - 閾値を左に動かす → 感度↑、特異度↓
   - 閾値を右に動かす → 感度↓、特異度↑

3. **有病率を変えてPPV/NPVの変化を確認**
   - 有病率が低い場合、PPV（陽性的中率）が大きく下がることを観察

---
## 評価指標の基礎知識

### 混同行列（Confusion Matrix）

二値分類の結果は、4つのカテゴリに分けられます：

```
                    予測: 陰性      予測: 陽性
実際: 正常          TN（真陰性）    FP（偽陽性）
実際: 異常          FN（偽陰性）    TP（真陽性）
```

- **TP (True Positive)**: 病気を正しく「陽性」と判定
- **TN (True Negative)**: 健康を正しく「陰性」と判定
- **FP (False Positive)**: 健康なのに「陽性」と誤判定（偽陽性）
- **FN (False Negative)**: 病気なのに「陰性」と誤判定（偽陰性）

### 主要な評価指標

| 指標 | 計算式 | 意味 |
|------|--------|------|
| 感度 (Sensitivity) | TP / (TP + FN) | 病気の人を正しく検出する割合 |
| 特異度 (Specificity) | TN / (TN + FP) | 健康な人を正しく除外する割合 |
| 正解率 (Accuracy) | (TP + TN) / 全体 | 全体の正解率 |
| 適合率 (Precision/PPV) | TP / (TP + FP) | 陽性予測の信頼度 |
| 陰性的中率 (NPV) | TN / (TN + FN) | 陰性予測の信頼度 |

---
## Step 1: 環境構築とデータ準備

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise.git /tmp/MedMNIST-Exercise -q
sys.path.insert(0, '/tmp/MedMNIST-Exercise')
sys.modules.pop('exercise_logic', None)
import exercise_logic
exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

In [ ]:
# 胸部X線データのロード（PneumoniaMNIST・肺炎検出）
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess(
    'pneumoniamnist'
)

print(f"✓ 訓練データ: {x_train.shape}")
print(f"✓ テストデータ: {x_test.shape}")
print(f"\nラベル: {info['label']}")
print(f"\n【データの偏り】")
print(f"  訓練データ: 正常 {int((y_train == 0).sum())}件, 肺炎 {int((y_train == 1).sum())}件")
print(f"  テストデータ: 正常 {int((y_test == 0).sum())}件, 肺炎 {int((y_test == 1).sum())}件")

---
## データセットのサンプル画像

まず、これから扱うデータセットがどのような画像なのか見てみましょう。

In [ ]:
# データセットのサンプル画像を表示
exercise_logic.show_sample_images(x_train, n_samples=10)

---
## Step 2: モデルの学習

第3回と同様に、二値分類モデルを学習します。

In [ ]:
# 二値分類モデルの構築と学習
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=1  # 二値分類: sigmoid出力
)

print("モデルを学習中...（約1-2分）")
history = model.fit(
    x_train, y_train, 
    epochs=5, 
    validation_split=0.1, 
    batch_size=128,
    verbose=1
)
print("\n✓ モデル学習が完了しました。")

---
## Step 3: ROC曲線とAUC

### ROC曲線とは？

ROC曲線（Receiver Operating Characteristic curve）は、分類器の性能を**閾値に依存しない形**で評価するグラフです。

- **X軸**: 偽陽性率（FPR = 1 - 特異度）
- **Y軸**: 真陽性率（TPR = 感度）
- **対角線**: ランダム分類器（コイン投げと同じ性能）
- **左上に近いほど良い**: 高感度かつ低偽陽性率

### AUC（Area Under the Curve）

AUCはROC曲線の下の面積で、0.5〜1.0の値を取ります。

| AUC | 解釈 |
|-----|------|
| 0.9-1.0 | 非常に良い（Excellent） |
| 0.8-0.9 | 良い（Good） |
| 0.7-0.8 | まずまず（Fair） |
| 0.6-0.7 | 不十分（Poor） |
| 0.5-0.6 | 失敗（Fail） |

In [ ]:
# ROC曲線の描画
roc_result = exercise_logic.plot_roc_curve(model, x_test, y_test, title="Pneumonia Detection - ROC Curve")

print(f"\n【結果】")
print(f"  AUC: {roc_result['auc']:.4f}")
print(f"  最適閾値（Youden Index基準）: {roc_result['optimal_threshold']:.4f}")
print(f"  最適点での感度: {roc_result['optimal_point'][1]:.4f}")
print(f"  最適点での特異度: {1 - roc_result['optimal_point'][0]:.4f}")

### ROC曲線の読み方

- **曲線が左上に膨らんでいる** → 良い分類器
- **曲線が対角線に近い** → ランダム分類と同程度
- **赤い点** → 最適な閾値での動作点（Youden Index最大化）

---
## Step 4: Precision-Recall曲線

### ROC曲線との使い分け

- **ROC曲線**: クラスバランスが取れている場合に適切
- **PR曲線**: 陽性クラスが少ない（不均衡）場合に適切

データのクラス分布を確認し、適切な評価曲線を選択しましょう。  
実際の臨床では**異常が少数派**のことが多いため、PR曲線の見方も知っておく価値があります。

In [ ]:
# Precision-Recall曲線の描画
pr_result = exercise_logic.plot_precision_recall_curve(model, x_test, y_test, title="Pneumonia Detection - PR Curve")

print(f"\n【結果】")
print(f"  Average Precision (AP): {pr_result['average_precision']:.4f}")

---
## Step 5: 閾値による評価指標の変化

### デフォルト閾値（0.5）での評価

まず、一般的に使われる閾値0.5での性能を確認します。

In [ ]:
# 閾値0.5での評価
result_default = exercise_logic.evaluate_at_threshold(model, x_test, y_test, threshold=0.5)

### 最適閾値での評価

Youden Indexで算出された最適閾値を使ってみましょう。

In [ ]:
# 最適閾値での評価
optimal_thresh = roc_result['optimal_threshold']
result_optimal = exercise_logic.evaluate_at_threshold(model, x_test, y_test, threshold=optimal_thresh)

### 複数閾値の比較

閾値を変えると、感度と特異度がどう変化するか確認しましょう。

In [ ]:
# 複数閾値での比較
exercise_logic.compare_thresholds(model, x_test, y_test, thresholds=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])

---
## Step 6: 目的に応じた閾値選択

### 医療における閾値選択の考え方

| 用途 | 重視する指標 | 閾値の傾向 |
|------|-------------|------------|
| スクリーニング検査 | 高感度（見逃し防止） | 低め |
| 確定診断 | 高特異度（誤診防止） | 高め |
| バランス重視 | Youden Index / F1 | 中程度 |

### 実践：目標感度95%を達成する閾値を探す

In [ ]:
# 感度95%を達成する閾値を探索
high_sens_result = exercise_logic.find_optimal_threshold(
    model, x_test, y_test, 
    strategy='sensitivity', 
    target_sensitivity=0.95
)

print("【スクリーニング用閾値（感度95%目標）】")
print(f"  閾値: {high_sens_result['threshold']:.4f}")
print(f"  感度: {high_sens_result['sensitivity']:.4f}")
print(f"  特異度: {high_sens_result['specificity']:.4f}")

In [ ]:
# この閾値での詳細評価
result_screening = exercise_logic.evaluate_at_threshold(
    model, x_test, y_test, 
    threshold=high_sens_result['threshold']
)

### 練習：特異度95%を達成する閾値を探してみよう

`strategy='specificity'` と `target_specificity=0.95` を使って、確定診断用の閾値を探してください。

In [ ]:
# 練習：特異度95%を達成する閾値を探索
# 以下のコードを完成させてください

high_spec_result = exercise_logic.find_optimal_threshold(
    model, x_test, y_test, 
    strategy='specificity',  # ← 'specificity' に変更
    target_specificity=0.95   # ← target_specificity を指定
)

print("【確定診断用閾値（特異度95%目標）】")
print(f"  閾値: {high_spec_result['threshold']:.4f}")
print(f"  感度: {high_spec_result['sensitivity']:.4f}")
print(f"  特異度: {high_spec_result['specificity']:.4f}")

---
## 考察課題

以下の点について考えてみましょう：

1. **閾値のトレードオフ**: 感度を上げると特異度が下がりました。この関係はなぜ生じるのでしょうか？

2. **スクリーニングと確定診断**: 胸部X線による肺炎診断で、スクリーニング段階と精密検査段階では、どちらの閾値設定が適切でしょうか？

3. **AUCの限界**: AUCが同じ0.85のモデルが2つあった場合、どちらが「良い」とは言えません。なぜでしょうか？

---
## まとめ

本講義で学んだ内容：

- **ROC曲線**: 閾値に依存しない分類器性能の評価
- **AUC**: ROC曲線下の面積、0.5（ランダム）〜1.0（完璧）
- **感度と特異度のトレードオフ**: 閾値を変えることで調整可能
- **目的に応じた閾値選択**:
  - スクリーニング → 高感度（見逃し防止）
  - 確定診断 → 高特異度（誤診防止）
- **PR曲線**: 不均衡データでの評価に有用

次回は「Webアプリ公開」を通じて、学習したモデルを社会に届ける方法を学びます。

---
## 考察課題の回答例

以下は考察課題に対する回答の一例です。これが唯一の正解ではなく、議論の出発点として活用してください。

### 1. 閾値のトレードオフが生じる理由

正常群と肺炎群の予測スコア分布が重なっているため、閾値を動かすと必ずトレードオフが生じる：

- 閾値を下げる → より多くを「陽性」と判定 → 感度↑、特異度↓
- 閾値を上げる → より多くを「陰性」と判定 → 感度↓、特異度↑
- 分布が完全に分離していれば（AUC=1.0）、トレードオフなく両方を最大化できる

### 2. スクリーニングと確定診断の閾値

一般的な設計指針：

- **スクリーニング（一次検査）**: 高感度を優先
  - 見逃しは許容しにくいため、偽陰性を最小化
  - 偽陽性は精密検査で除外できる
- **確定診断（精密検査）**: 高特異度を優先
  - 治療の判断に使うため、誤診のコストが高い
  - 追加検査や培養検査で確認できることが多い

### 3. AUCの限界

AUCが同じでも、ROC曲線の形状が異なる場合がある：

- 高感度域で優れたモデル vs 高特異度域で優れたモデル
- 使用目的（スクリーニング or 確定診断）により、適したモデルが異なる
- AUCは「全体的な性能」を示すが、「特定の動作点での性能」は別途確認が必要
- 臨床的には、目標とする感度・特異度を満たすかどうかで判断することが多い

---
## 発展的な学習（技術的詳細に興味のある方へ）

この講義では、技術的な詳細を `exercise_logic.py` に分離しています。
より深く学びたい方は、以下の関数のソースコードを参照してください。

### この講義で使用した主要関数

| 関数名 | 機能 | 技術的なポイント |
|--------|------|------------------|
| `initialize_environment()` | 環境セットアップ | Colab/Local判定、GPU設定 |
| `load_and_preprocess()` | データ読み込み・前処理 | MedMNISTの構造、チャンネル変換 |
| `show_sample_images()` | サンプル画像表示 | データセットの概観把握 |
| `build_model()` | CNNモデル構築 | 二値分類用sigmoid出力 |
| `plot_roc_curve()` | ROC曲線描画・AUC計算 | sklearn.metrics.roc_curve, Youden Index |
| `plot_precision_recall_curve()` | PR曲線描画 | 不均衡データでの評価 |
| `find_optimal_threshold()` | 最適閾値探索 | 戦略に応じた閾値選択 |
| `evaluate_at_threshold()` | 閾値指定での評価 | 混同行列からの指標計算 |
| `compare_thresholds()` | 複数閾値の比較 | 一覧表での性能比較 |

### ソースコードの参照方法

`exercise_logic.py` はGitHubリポジトリで公開しています：

https://github.com/kshimoji8/MedMNIST-Exercise/blob/main/exercise_logic.py

各関数には詳細な技術解説をdocstring（関数冒頭のコメント）として記載しています。